### OJO 👁️👁️
con la base de CHATGPT >>
 comparar metodología con el curso

Dejo un requirements.txt para que corra este pipeline en: <code>/entorno_conda/tfm-ml/requirements.txt</code>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, explained_variance_score

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.ensemble import ExtraTreesRegressor, AdaBoostRegressor, HistGradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import FunctionTransformer
from skops.io import dump, load, get_untrusted_types
from zipfile import ZIP_DEFLATED

In [ ]:

# 1. Carga de datos
completo = pd.read_csv('transform_listings.csv')
df = completo[['neighbourhood_group_cleansed','bedrooms', 'bathrooms', 'price','latitude','longitude']]
df


In [ ]:
# 2.1 No hago preprocesado porque ya lo hice en el notebook de transform_listings

# más adelante filtro precios outliers para mejorar los resultados de los modelos

In [ ]:
# Histogramay bigotes sin quitar outliers

fig, axes = plt.subplots(1, 2, figsize=(12, 3))
df['price'].plot(kind='hist', ax=axes[0], bins=1000)
# df['price'].plot(kind='hist', ax=axes[0], bins=int(np.sqrt(len(df['price']))))
axes[0].set_title('Histograma de Precios')
axes[0].set_xlabel('Precio')
axes[0].set_ylabel('Frecuencia')
df['price'].plot(kind='box', ax=axes[1], vert=False)
axes[1].set_title('Gráfico de Bigotes de Precios')
axes[1].set_xlabel('Precio')
plt.tight_layout()
plt.show()

In [ ]:
# # # # Filtro los outliers utilizando el rango intercuartílico (IQR): 
# # # ¿Mejoraría algo con z-score?
Q1  = df['price'].quantile(0.25)
Q3  = df['price'].quantile(0.75)
IQR = Q3 - Q1
IQR
df = df[(df['price']>=Q1 - 1.5 * IQR) & (df['price']<=Q3 + 1.5 * IQR)]

In [ ]:
# Histograma y bigotes con outliers quitados (método IQR)

fig, axes = plt.subplots(1, 2, figsize=(12, 3))
df['price'].plot(kind='hist', ax=axes[0], bins=int(np.sqrt(len(df['price']))))
axes[0].set_title('Histograma de Precios')
axes[0].set_xlabel('Precio')
axes[0].set_ylabel('Frecuencia')
df['price'].plot(kind='box', ax=axes[1], vert=False)
axes[1].set_title('Gráfico de Bigotes de Precios')
axes[1].set_xlabel('Precio')
plt.tight_layout()
plt.show()

In [ ]:
df.info()

In [ ]:
# densifier = FunctionTransformer(lambda X: X.toarray() if hasattr(X, "toarray") else X)

# 1. Carga y limpieza
df = df.dropna(subset=['bedrooms','bathrooms','neighbourhood_group_cleansed','price'])

# 2. Separar X/y
X = df[['bedrooms','bathrooms','neighbourhood_group_cleansed']]
y = df['price']

# 3. Train/Test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# 4. Preprocesador: escala numéricos + one-hot para distrito
numeric_feats = ['bedrooms','bathrooms']
cat_feats     = ['neighbourhood_group_cleansed']

preprocessor = ColumnTransformer([
    ('num',   StandardScaler(),    numeric_feats),
    ('cat',   OneHotEncoder(drop='first', handle_unknown="ignore", sparse_output=False), cat_feats),
])

# 5. Definir pipelines para cada modelo
models = {
    'LinearRegression': Pipeline([('prep', preprocessor),
                                  ('reg',  LinearRegression())]),
    'Ridge':            Pipeline([('prep', preprocessor),
                                  ('reg',  Ridge(alpha=1.0, random_state=42))]),
    'Lasso':            Pipeline([('prep', preprocessor),
                                  ('reg',  Lasso(alpha=0.1, random_state=42))]),
    'ElasticNet':       Pipeline([('prep', preprocessor),
                                  ('reg',  ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42))]),
    'KNeighbors':       Pipeline([('prep', preprocessor),
                                  ('reg',  KNeighborsRegressor(n_neighbors=5))]),
    'SVR':              Pipeline([('prep', preprocessor),
                                  ('reg',  SVR(kernel='rbf', C=1.0))]),
    'RandomForest':     Pipeline([('prep', preprocessor),
                                  ('reg',  RandomForestRegressor(n_estimators=100, random_state=42))]),
    'GradientBoosting': Pipeline([('prep', preprocessor),
                                  ('reg',  GradientBoostingRegressor(
                                      n_estimators=100, learning_rate=0.1, 
                                      max_depth=3, random_state=42))]),
    'ExtraTrees': Pipeline([('prep', preprocessor),
                            ('reg', ExtraTreesRegressor(n_estimators=100, random_state=42))]),
    'AdaBoost': Pipeline([('prep', preprocessor),
                          ('reg', AdaBoostRegressor(n_estimators=100, random_state=42))]),
    'HistGradientBoosting': Pipeline([('prep', preprocessor),
                                    #   ('densify',densifier),
                                      ('reg', HistGradientBoostingRegressor(max_iter=100, random_state=42))]),
    'MLPRegressor': Pipeline([('prep', preprocessor),
                              ('reg', MLPRegressor(hidden_layer_sizes=(100,), max_iter=500, random_state=42))]),
}

# 6. Definir métricas para cross_validate
scoring = {
    'MAE':   'neg_mean_absolute_error',
    'RMSE':  'neg_root_mean_squared_error',
    'R2':    'r2',
    'MAPE':  'neg_mean_absolute_percentage_error',
    'EVS':   'explained_variance',
}

# 7. Ejecutar validación cruzada en train
results = {}
for name, pipe in models.items():
    cv = cross_validate(pipe, X_train, y_train,
                        cv=5, scoring=scoring,
                        return_train_score=False)
    # invertir signos de las métricas negativas
    mae_mean  = -cv['test_MAE'].mean()
    rmse_mean = -cv['test_RMSE'].mean()
    mape_mean = -cv['test_MAPE'].mean()
    results[name] = {
        'MAE':  mae_mean,
        'RMSE': rmse_mean,
        'R2':   cv['test_R2'].mean(),
        'MAPE': mape_mean,
        'EVS':  cv['test_EVS'].mean(),
    }

# 8. Mostrar comparativa ordenada por MAE
results_df = pd.DataFrame(results).T.sort_values('MAE')
display(results_df.round(3))


He visto esto respecto a métricas:
https://medium.com/@nicolasarrioja/m%C3%A9tricas-en-regresi%C3%B3n-5e5d4259430b


Exportamos el modelo para utilizarlo después en otros DF

In [ ]:
# 0. Tenemos results_df ya ordenado por MAE ascendente
best_name, best_pipe = results_df.index[0], models[results_df.index[0]]

# 1. Reentrenar con TODO el dataset de entrenamiento+validación
X_dev = pd.concat([X_train, X_test], axis=0)
y_dev = pd.concat([y_train, y_test], axis=0)
best_pipe.fit(X_dev, y_dev)

# 2. Persistir de forma segura
# dump(best_pipe, f"{best_name}.skops", compression="lzma") ## Esta compresión me da problemas, creo que debo instalar algo en el entorno.
dump(best_pipe, f"pers_models/{best_name}.skops", compression=ZIP_DEFLATED, compresslevel=9)

# 3. Evaluar en el mismo X_dev, y_dev (ya que no tenemos un test_final aparte)
y_pred_dev = best_pipe.predict(X_dev)
mae_dev = mean_absolute_error(y_dev, y_pred_dev)
rmse_dev = np.sqrt(mean_squared_error(y_dev, y_pred_dev))
r2_dev = r2_score(y_dev, y_pred_dev)
mape_dev = mean_absolute_percentage_error(y_dev, y_pred_dev)
evs_dev = explained_variance_score(y_dev, y_pred_dev)
# print(f"MAE del mejor modelo ({best_name}) reentrenado en X_dev: {mae_dev:.3f}")
# print(f"RMSE del mejor modelo ({best_name}) reentrenado en X_dev: {rmse_dev:.3f}")
# print(f"R2 del mejor modelo ({best_name}) reentrenado en X_dev: {r2_dev:.3f}")
# print(f"MAPE del mejor modelo ({best_name}) reentrenado en X_dev: {mape_dev:.3f}")
# print(f"EVS del mejor modelo ({best_name}) reentrenado en X_dev: {evs_dev:.3f}")

# Guardar todas las métricas del modelo final
final_metrics = {
    'MAE': mae_dev,
    'RMSE': rmse_dev,
    'R2': r2_dev,
    'MAPE': mape_dev,
    'EVS': evs_dev,
}
# print("\nMétricas finales del modelo reentrenado:")
print(pd.Series(final_metrics).rename(best_name).to_frame().T.round(3))


Para inferencia de otros datos con el mismo modelo:

In [ ]:
model_path = f"pers_models/{best_name}.skops"
untrusted = get_untrusted_types(file=model_path) #Esto es una ñapa, porque debo confiar en la función lambda de densifier
print("Untrusted:", untrusted)
model = load(model_path, trusted=untrusted)
# pred = model.predict(new_data)